In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Analytic-olympiad/train.csv
/kaggle/input/Analytic-olympiad/test.csv


In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import tensorflow as tf

In [3]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.feature_selection import VarianceThreshold

#Implementingthe Transformer class
class drop_col(TransformerMixin):
    def __init__(self,threshold=0.3):
        self.threshold=threshold
    def fit(self,X,y=None):
        col_vars=X.var()
        self.col_to_drop=col_vars[col_vars<self.threshold].index
        return self
    def transform(self,X):
        assert self.col_to_drop is not None, 'Drop_col error, must be fitted before predict'
        X.drop(self.col_to_drop, axis=1, inplace=True)
        return X

In [4]:
#Reading the dataset
FILEPATH="/kaggle/input/Analytic-olympiad"
df_train=pd.read_csv(os.path.join(FILEPATH,'train.csv'))
df_test=pd.read_csv(os.path.join(FILEPATH,'train.csv'))

#Defining the features
cat_cols=df_train.select_dtypes(include="object").columns.to_list()
num_cols=df_train.select_dtypes(include=["int64","float64"]).columns.to_list()[:-2]
target_cols=df_train.columns.to_list()[-2:]
#df_train.isna().sum() #There are no null values

In [5]:
#Removing columns with low variance
remove_col=drop_col(1)
df_train=pd.concat([remove_col.fit_transform(df_train[num_cols]),df_train[cat_cols+target_cols]])
df_test=pd.concat([remove_col.transform(df_test[num_cols]),df_test[cat_cols+target_cols]])